# Parqet Converter V2
Just a simple script to convert EquatePlus export to a csv compatible with Parqet

In [ ]:
monthsBack = 1
isin = 'CH0012032048'
currency = 'CHF'

## Imports:

In [ ]:
from datetime import date
import openpyxl
import pandas as pd
import warnings
import yaml

In [ ]:
pd.set_option('display.max_rows', None)

# Load Configuration:

In [ ]:
configFile = 'ParqetConverterConfig.yaml'
with open('ParqetConverterConfig.yaml') as f:
    config = yaml.safe_load(f)

loadFilename = config['loadFilename']
monthsBack = config['monthsBack']
isin = config['isin']
currency = config['currency']

print(f'Loaded {len(config)} configuration items from {configFile}.')

## Open and clean the Excel-Sheet:
openpyxl prints some warnings with the default setting which can be ignored

In [ ]:
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    # eppd = EquatePlusPortfolioDetails
    eppd = pd.read_excel(loadFilename, engine="openpyxl")

Delete the header rows (top 5)

In [ ]:
eppd = eppd.iloc[4:].reset_index(drop=True)

Convert the top row to column headers:

In [ ]:
column_headers = eppd.iloc[0]

eppd_cleaned = eppd[1:]
eppd_cleaned.columns = column_headers

print(f'Loaded PortfoioDetails with {len(eppd)} rows.')

Remove columns with unnecessary metadata

In [ ]:
# remove unnecessary columns
remove_columns = ['Plan', 
                  'Instrumententyp', 
                  'Instrument', 
                  'Marktpreis',
                  'Ablaufdatum', 
                  'Ausstehende Menge', 
                  'Verfügbare Menge', 
                  'Geschätzter aktueller ausstehender Wert', 
                  'Geschätzter aktueller verfügbarer Wert']
eppd_c_relevant_cols = eppd_cleaned.drop(columns= remove_columns)

print(f'Dropped {len(remove_columns)} columns.')

Rename Columns

In [ ]:
# rename necessary columns
rename_columns = {'Allokationsdatum' : 'date',
                  'Beitragsart' : 'type',
                  'Ausübungspreis / Einstandspreis' : 'price',
                  'Zugewiesene Menge' : 'shares',
                  'Verfügbar ab' : 'availability'
                  }
eppd_c_relevant_cols.rename(columns= rename_columns, inplace=True)

print(f'Renamed {len(rename_columns)} columns.')

Rename column values

In [ ]:
pd.set_option('future.no_silent_downcasting', True)
# rename columns
replace_values = [
    #(original, new)
    ('Kauf', 'Buy'),
    ('Unternehmensbeitrag', 'Buy'),
]

for original, newValue in replace_values:
    eppd_c_relevant_cols.replace(original, newValue, inplace= True)

print(f'Renamed {len(replace_values)} colum values.')
print('Data cleaning complete.')

# Identify relevant data
Filter for time

In [ ]:
print('Start data processing.')

In [ ]:
# filter by monthsBack
if(monthsBack > 0):
    eppd_c_relevant_cols['tmpDate'] = eppd_c_relevant_cols['date']
    eppd_c_relevant_cols['tmpDate'] = pd.to_datetime(eppd_c_relevant_cols['tmpDate'])
    today = pd.Timestamp.now()
    month = today.floor('d') + pd.offsets.MonthEnd(0) - pd.offsets.MonthBegin(monthsBack)
    month
    eppd_c_r_cols = eppd_c_relevant_cols[(eppd_c_relevant_cols['tmpDate'] > month)]
    eppd_c_rc = eppd_c_r_cols.drop(columns=['tmpDate'])
else:
    eppd_c_rc = eppd_c_relevant_cols

print(f'Continuing with {len(eppd_c_rc)} rows.')

## Identify and handle dividends
Dividends are buys which are immedeatly available

In [ ]:
dividend_rows = eppd_c_rc[eppd_c_rc['date'] == eppd_c_rc['availability']].copy()
dividend_rows['type'] = 'Dividend'
eppd_c_rc = pd.concat([eppd_c_rc, dividend_rows])
eppd_c_rc = eppd_c_rc.sort_values(by='date')
eppd_c_rc = eppd_c_rc.drop(columns=['availability'])

print(f'Found {len(dividend_rows)} dividends')

# Add metadata and order dataframe

In [ ]:
# insert metadata
insert_columns = [
    #(column, value, loc)
    ('fee', 0, 1),
    ('isin', isin, 2),
    ('tax', 0, 5),
    ('currency', currency, 7)]
for column, value, loc in insert_columns:
    eppd_c_rc.insert(column=column, value=value, loc=loc)

print ('Added metadata')

In [ ]:
# reorder columns
column_order = [
    'date',
    'fee',
    'isin',
    'shares',
    'price',
    'tax',
    'type',
    'currency'
]
eppd_c_rc = eppd_c_rc[column_order]

print('Reordered columns to: ' + str(column_order))

# Export

In [ ]:
eppd_c_rc['date'] = pd.to_datetime(eppd_c_rc['date'])
eppd_c_rc

In [ ]:
# export
fileName = 'PortfolioDetails-' + str(date.today()) + '.csv'
eppd_c_rc.to_csv(fileName, sep=';')
print(f'Export to {fileName} complete.')